In [12]:
""" 

Extractor.ipynb

This Jupyter notebook contains a Python class called HeaderSelector. The HeaderSelector class is used to read an Excel spreadsheet, display the headers in a user-friendly, natural language format, 
and allow the user to select multiple headers from a list. The selected headers are then used to create a new DataFrame, which is saved to a new Excel file. 

The class also includes methods to apply custom CSS styles to the widgets and the container of the widgets, providing an overall dark aesthetic.

The notebook uses ipywidgets for creating interactive widgets, pandas for handling data, and the openpyxl library for reading and writing Excel files.

This script follows the Pylint format and adheres to standard Python conventions for readability and maintainability.

Author: [Ara Alexandrian] Date: [1/24/2024] 

"""

# Importing necessary libraries
import configparser
import pandas as pd
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display

In [15]:
class HeaderSelector:
    def __init__(self, spreadsheet_path):
        # Load the spreadsheet into a DataFrame
        self.df = pd.read_excel(spreadsheet_path)

        # Strip any leading/trailing white spaces from column names
        self.df.columns = self.df.columns.str.strip()

        # Convert the column headers to natural language and keep a mapping of the original headers to the updated headers
        self.original_headers = self.df.columns.tolist()
        self.headers_mapping = {self.convert_to_natural_language(header): header for header in self.original_headers}

        # Remove the "Local_ID" from the list of headers
        self.headers = [header for header in self.headers_mapping.keys() if header != "Local_ID"]

        # Initialize an empty list to store selected headers
        self.selected_headers = []

        # Create a SelectMultiple widget with the headers as options
        self.listbox = widgets.SelectMultiple(options=self.headers, layout=Layout(height='300px', width='500px'))

        # Create a Save button
        self.save_button = widgets.Button(description='Save', button_style='success')

        # Bind the on_save_clicked method to the button's on_click event
        self.save_button.on_click(self.on_save_clicked)

        # Create a container for the listbox and the save button
        self.container = widgets.VBox([self.listbox, self.save_button])

    def on_save_clicked(self, button):
        # Get the selected headers
        self.selected_headers = list(self.listbox.value)

        # Convert the selected headers back to their original format
        original_selected_headers = [self.headers_mapping[header] for header in self.selected_headers]

        # Check if all selected headers are in the DataFrame
        if set(original_selected_headers).issubset(self.original_headers):
            # Create a new DataFrame with only the selected headers
            new_df = self.df[original_selected_headers]

            # Rename the columns in the new DataFrame to the updated format
            new_df.columns = self.selected_headers

            # Save the new DataFrame to an Excel file
            new_df.to_excel('extracted.xlsx', index=False)

            print("Success! The selected headers have been saved to extracted.xlsx")
        else:
            print("Error: Some selected headers are not in the DataFrame")

    def convert_to_natural_language(self, header):
        # Convert the header to natural language
        converted_header = header.split('.')[-1]
        return converted_header


In [17]:
# Read the config.ini file
config = configparser.ConfigParser()
config.read('config.ini')

header_selector = HeaderSelector(config['SPREADSHEET']['FILE_PATH'])
print("DataFrame headers:", header_selector.headers)
display(header_selector.container)

DataFrame headers: ['Event Number', 'Date_Time_Submitted', 'Classification', 'Location_Sub', 'Narrative', 'Tx_Technique', 'Tag/Topic', 'Reporter_Name', 'Discoverer_Role', 'Narrative_Supplemental', 'Event_Title']


Success! The selected headers have been saved to extracted.xlsx
